<p style="font-size:30px; text-align:center; line-height:120%">
    <br> 
        <b>
        COMS 4995 Applied ML
            Homework 4 
        <br></br>
            Wine Reviews: Task 1
        <br></br>
        </b> 
    <br> 
</p>
<p style="font-size:18px; text-align:left; line-height:120%">
    <br> 
        <b>
        Name: Kirit Dhillon, Sagar Lal
        </b>
    <br> 
        <b>
        Uni: ksd2142, sl3946
        </b>
</p>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loanding and Exploration

In [5]:
data = pd.read_csv("winemag-data-130k-v2.csv")

# Remove non-usefule columns
data = data.drop(['taster_name', 'taster_twitter_handle'], axis=1)

In [6]:
print(data.dtypes)
print(data.shape)

Unnamed: 0       int64
country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
title           object
variety         object
winery          object
dtype: object
(129971, 12)


In [9]:
text_data_x = data[['description']]
non_text_data_x = data.drop(['description', 'points'], axis=1)

data_y = data['points']

In [11]:
print(text_data_x.shape)
print(non_text_data_x.shape)
print(data_y.shape)

(129971, 1)
(129971, 10)
(129971,)


## 1.1: Baseline model using non-text features

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [13]:
X_trainval, X_test, y_trainval, y_test = train_test_split(non_text_data_x, data_y)
continuous_features = ['price']
categorical_features = ['country', 'title', 'designation', 'province', 'region_1', 'region_2',
                        'variety', 'winery']

In [14]:
base_categorical_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = 'constant', fill_value = "missing")),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

base_continuous_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = "median")),
                                         ('scaler', StandardScaler())])

base_continuous_and_categorical_preprocessor = ColumnTransformer(
    transformers=[
        ('num', base_continuous_transformer, continuous_features),
        ('cat', base_categorical_transformer, categorical_features)
    ])

In [16]:
# Baseline Ridge
base_ridge_clf = Pipeline(steps=[('preprocessor', base_continuous_and_categorical_preprocessor),
                                 ('classifier', Ridge())])

base_ridge_scores = cross_val_score(base_ridge_clf, X_trainval, y_trainval, cv=5)

print("Baseline Ridge with only CV: ", np.mean(base_ridge_scores))

Baseline Ridge with only CV:  0.50319153324433


In [18]:
ridge_pipeline = Pipeline(steps=[('preprocessor', base_continuous_and_categorical_preprocessor),
                                ('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=5, return_train_score=True)
grid.fit(X_trainval, y_trainval)

print(("Baseline Ridge score with GridSearchCV %.2f"
       %grid.score(X_test, y_test)))
print("Best param:", grid.best_params_)

Baseline Ridge score with GridSearchCV 0.53
Best param: {'regressor__alpha': 1}


## 1.2: Text-based model based on:
- Bag-of-Words
- Linear Model

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

X_trainval, X_test, y_trainval, y_test = train_test_split(text_data_x, data_y)

In [20]:
vect = CountVectorizer(stop_words="english", max_features=1000)

X_trainval_transform = vect.fit_transform(X_trainval['description'])
X_test_transform = vect.transform(X_test['description'])

In [23]:
print(X_trainval_transform.shape)
print(X_test_transform.shape)

(97478, 1000)
(32493, 1000)


In [24]:
ridge_pipeline = Pipeline(steps=[('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(X_trainval_transform, y_trainval)

print(("Text-based Rigde score with GridSearchCV %.2f"
       %grid.score(X_test_transform, y_test)))
print("Best param:", grid.best_params_)

Text-based Rigde score with GridSearchCV 0.61
Best param: {'regressor__alpha': 10}


## 1.3: More text-based models:
- n-grams
- characters
- tf-idf
- other ways to tune BoW

## 1.4: Text and non-text features hybrid model